In [10]:
import pandas as pd
import plotly.graph_objects as go
from collections import deque
from IPython.display import display, HTML

In [11]:
trades_wn_dn1 = pd.read_csv('trades_round_1_day_-1_wn.csv', delimiter=';')
trades_wn_dn2 = pd.read_csv('trades_round_1_day_-2_wn.csv', delimiter=';')
trades_wn_d0 = pd.read_csv('trades_round_1_day_0_wn.csv', delimiter=';')

trades_r3_d0 = pd.read_csv('trades_round_3_day_0_wn.csv', delimiter=';')
trades_r3_d1 = pd.read_csv('trades_round_3_day_1_wn.csv', delimiter=';')
trades_r3_d2 = pd.read_csv('trades_round_3_day_2_wn.csv', delimiter=';')

trades_r4_d1 = pd.read_csv('trades_round_4_day_1_wn.csv', delimiter=';')
trades_r4_d2 = pd.read_csv('trades_round_4_day_2_wn.csv', delimiter=';')
trades_r4_d3 = pd.read_csv('trades_round_4_day_3_wn.csv', delimiter=';')

In [16]:
def plot_trades(df, trader_name, product_name):
    buys = df[(df['buyer'] == trader_name) & (df['symbol'] == product_name)]
    sells = df[(df['seller'] == trader_name) & (df['symbol'] == product_name)]

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=buys['timestamp'], y=buys['price'], mode='markers', name='Buys', marker=dict(color='green', size=10, opacity=0.5)))
    fig.add_trace(go.Scatter(x=sells['timestamp'], y=sells['price'], mode='markers', name='Sells', marker=dict(color='red', size=10, opacity=0.5)))

    fig.update_layout(title=f'Trades for {trader_name} on {product_name}',
                      xaxis_title='Timestamp',
                      yaxis_title='Price',
                      legend_title='Transaction Type')
    fig.show()


In [17]:
def plot_realized_pl(df, trader_name, product_name):
    buys = df[(df['buyer'] == trader_name) & (df['symbol'] == product_name)].copy()
    buys['type'] = 'buy'
    buys['quantity'] = buys['quantity']
    
    sells = df[(df['seller'] == trader_name) & (df['symbol'] == product_name)].copy()
    sells['type'] = 'sell'
    sells['quantity'] = -sells['quantity']  # Treat sells as negative for uniform handling
    
    transactions = pd.concat([buys, sells])
    transactions.sort_values('timestamp', inplace=True)

    positions = deque()
    realized_pl = []
    pl_total = 0

    for _, transaction in transactions.iterrows():
        transaction_quantity = transaction['quantity']
        transaction_price = transaction['price']
        transaction_timestamp = transaction['timestamp']

        while transaction_quantity != 0:
            if not positions:
                positions.append({'quantity': transaction_quantity, 'price': transaction_price})
                break

            position = positions[0]
            if (position['quantity'] > 0 and transaction_quantity < 0) or (position['quantity'] < 0 and transaction_quantity > 0):
                quantity_to_close = min(abs(transaction_quantity), abs(position['quantity']))
                pl = quantity_to_close * (abs(transaction_price) - abs(position['price']))
                if position['quantity'] < 0:
                    pl = -pl

                pl_total += pl
                realized_pl.append({'timestamp': transaction_timestamp, 'P&L': pl_total})

                position['quantity'] += transaction_quantity
                if position['quantity'] == 0:
                    positions.popleft()  # Remove closed position
                transaction_quantity += quantity_to_close if transaction_quantity < 0 else -quantity_to_close
            else:
                positions.append({'quantity': transaction_quantity, 'price': transaction_price})
                break

    pl_df = pd.DataFrame(realized_pl)
    if not pl_df.empty:
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=pl_df['timestamp'], y=pl_df['P&L'], mode='lines+markers', name='Realized P&L'))
        fig.update_layout(title=f'Realized P&L for {trader_name} on {product_name}',
                          xaxis_title='Timestamp',
                          yaxis_title='Profit and Loss',
                          legend_title='P&L')
        fig.show()
    else:
        print("No completed transactions to calculate realized P&L.")


In [18]:
def analyze_trades(df):
    def get_unique_symbols(df):
        return df['symbol'].unique()
    
    def get_unique_traders(df):
        buyers = df['buyer'].unique()
        sellers = df['seller'].unique()
        return set(buyers).union(set(sellers))
    
    def get_buys(df, trader_name):
        return df[df['buyer'] == trader_name]

    def get_sells(df, trader_name):
        return df[df['seller'] == trader_name]

    unique_traders = get_unique_traders(df)
    unique_symbols = get_unique_symbols(df)

    for trader in unique_traders:
        for symbol in unique_symbols:
            print(f"\nTrader: {trader}, Symbol: {symbol}")
            buys = get_buys(df, trader)
            sells = get_sells(df, trader)

            buys = buys[buys['symbol'] == symbol]
            sells = sells[sells['symbol'] == symbol]

            if not buys.empty:
                print("Buys:")
                display(buys)
            else:
                print("No buys for this product.")

            if not sells.empty:
                print("Sells:")
                display(sells)
            else:
                print("No sells for this product.")

            if not buys.empty or not sells.empty:
                plot_trades(df, trader, symbol) 

            if not buys.empty and not sells.empty:
                plot_realized_pl(df, trader, symbol)
            else:
                print("Not enough data to calculate realized P&L.")

In [ ]:
# copypaste into the cell below to see 1 at a time
"""
analyze_trades(trades_wn_dn2)
analyze_trades(trades_wn_dn1)
analyze_trades(trades_wn_d0)

analyze_trades(trades_r3_d0)
analyze_trades(trades_r3_d1)
analyze_trades(trades_r3_d2)

analyze_trades(trades_r4_d1)
analyze_trades(trades_r4_d2)
analyze_trades(trades_r4_d3)
"""

In [19]:
analyze_trades(trades_wn_dn1)


Trader: Valentina, Symbol: STARFRUIT
Buys:


,timestamp,buyer,seller,symbol,currency,price,quantity
9,2400,Valentina,Remy,STARFRUIT,SEASHELLS,5038.0,1
15,3100,Valentina,Remy,STARFRUIT,SEASHELLS,5037.0,2
16,3200,Valentina,Remy,STARFRUIT,SEASHELLS,5037.0,1
21,3800,Valentina,Amelia,STARFRUIT,SEASHELLS,5037.0,14
22,3800,Valentina,Remy,STARFRUIT,SEASHELLS,5037.0,1
...,...,...,...,...,...,...,...
5352,996000,Valentina,Vinnie,STARFRUIT,SEASHELLS,5043.0,6
5358,996500,Valentina,Vinnie,STARFRUIT,SEASHELLS,5041.0,4
5359,997200,Valentina,Remy,STARFRUIT,SEASHELLS,5039.0,1
5360,997400,Valentina,Ruby,STARFRUIT,SEASHELLS,5039.0,4


Sells:


,timestamp,buyer,seller,symbol,currency,price,quantity
0,0,Amelia,Valentina,STARFRUIT,SEASHELLS,5047.0,13
1,0,Remy,Valentina,STARFRUIT,SEASHELLS,5047.0,1
13,3000,Remy,Valentina,STARFRUIT,SEASHELLS,5045.0,1
30,4800,Remy,Valentina,STARFRUIT,SEASHELLS,5044.0,2
31,5300,Remy,Valentina,STARFRUIT,SEASHELLS,5042.0,1
...,...,...,...,...,...,...,...
5324,991700,Remy,Valentina,STARFRUIT,SEASHELLS,5049.0,1
5345,995000,Remy,Valentina,STARFRUIT,SEASHELLS,5047.0,1
5356,996200,Adam,Valentina,STARFRUIT,SEASHELLS,5044.0,3
5363,998900,Remy,Valentina,STARFRUIT,SEASHELLS,5043.0,1



Trader: Valentina, Symbol: AMETHYSTS
Buys:


...

,timestamp,buyer,seller,symbol,currency,price,quantity
162,31300,Valentina,Ruby,STARFRUIT,SEASHELLS,5038.0,13
180,34200,Valentina,Ruby,STARFRUIT,SEASHELLS,5034.0,9
262,51300,Valentina,Ruby,STARFRUIT,SEASHELLS,5030.0,13
300,58400,Valentina,Ruby,STARFRUIT,SEASHELLS,5029.0,15
346,68000,Valentina,Ruby,STARFRUIT,SEASHELLS,5026.0,6
...,...,...,...,...,...,...,...
5183,966700,Vladimir,Ruby,STARFRUIT,SEASHELLS,5045.0,2
5184,966700,Vinnie,Ruby,STARFRUIT,SEASHELLS,5045.0,1
5204,972100,Valentina,Ruby,STARFRUIT,SEASHELLS,5042.0,12
5351,995700,Valentina,Ruby,STARFRUIT,SEASHELLS,5040.0,12



Trader: Ruby, Symbol: AMETHYSTS
Buys:


,timestamp,buyer,seller,symbol,currency,price,quantity
76,15300,Ruby,Vladimir,AMETHYSTS,SEASHELLS,10002.0,1
134,25700,Ruby,Vinnie,AMETHYSTS,SEASHELLS,10004.0,2
149,29000,Ruby,Valentina,AMETHYSTS,SEASHELLS,10005.0,8
164,31900,Ruby,Valentina,AMETHYSTS,SEASHELLS,10005.0,2
274,52900,Ruby,Valentina,AMETHYSTS,SEASHELLS,10005.0,6
...,...,...,...,...,...,...,...
5214,973900,Ruby,Vinnie,AMETHYSTS,SEASHELLS,10000.0,5
5238,977800,Ruby,Vinnie,AMETHYSTS,SEASHELLS,10004.0,2
5239,977800,Ruby,Valentina,AMETHYSTS,SEASHELLS,10004.0,1
5246,978100,Ruby,Remy,AMETHYSTS,SEASHELLS,9998.0,1


Sells:


,timestamp,buyer,seller,symbol,currency,price,quantity
24,4200,Vinnie,Ruby,AMETHYSTS,SEASHELLS,9996.0,2
55,11100,Valentina,Ruby,AMETHYSTS,SEASHELLS,9998.0,4
56,11100,Vladimir,Ruby,AMETHYSTS,SEASHELLS,9998.0,3
90,16800,Valentina,Ruby,AMETHYSTS,SEASHELLS,9995.0,5
104,21300,Vinnie,Ruby,AMETHYSTS,SEASHELLS,9996.0,1
...,...,...,...,...,...,...,...
5216,974100,Valentina,Ruby,AMETHYSTS,SEASHELLS,9995.0,7
5226,976800,Vladimir,Ruby,AMETHYSTS,SEASHELLS,9998.0,2
5227,976800,Remy,Ruby,AMETHYSTS,SEASHELLS,9998.0,1
5243,978000,Vinnie,Ruby,AMETHYSTS,SEASHELLS,9996.0,2
